In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')




In [11]:
os.getcwd()

'/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/Behavior_Scorecard-DATA-223'

In [14]:
path = "./2. Code/Classes"
os.chdir(path)

from Preprocessing_Application_N import DataPreprocessor

In [3]:

server = 'reporting-db.nystartfinans.net'
database = 'reporting-db'
username = 'admin'
password = 'Tnb1tr9SNUgpJhQAc1lt'
driver = '{ODBC Driver 17 for SQL Server}'  # This is an example for SQL Server, adjust according to your database and installed ODBC driver


In [6]:
import sqlalchemy as sa
import pandas as pd

# Your example DataFrame
data = {
    'Title': ['The Great Gatsby', 'To Kill a Mockingbird', '1984', 'Pride and Prejudice', 'The Catcher in the Rye'],
    'Author': ['F. Scott Fitzgerald', 'Harper Lee', 'George Orwell', 'Jane Austen', 'J.D. Salinger'],
    'Publication Year': [1925, 1960, 1949, 1813, 1951],
    'Genre': ['Novel', 'Fiction', 'Dystopian', 'Novel', 'Fiction']
}
books_df = pd.DataFrame(data)


# Update the driver to 'ODBC Driver 17 for SQL Server' for Native Client 17
# This assumes you're using ODBC Driver 17, which is the usual driver used with SQL Server Native Client 17 installations
engine = sa.create_engine(f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server')

# Upload DataFrame to SQL table
books_df.to_sql('ECLInput_play', con=engine, index=False, if_exists='replace', schema='nystart')


5

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')



################### GET PAYMENT DATA ##########################

path = "../2. Code/Classes"
os.chdir(path)

from Preprocessing_Application_N import DataPreprocessor

print('hej')


server = 'reporting-db.nystartfinans.net'
database = 'reporting-db'
username = 'admin'
password = 'Tnb1tr9SNUgpJhQAc1lt'
driver = '{ODBC Driver 17 for SQL Server}'  # This is an example for SQL Server, adjust according to your database and installed ODBC driver


# Initialize DataPreprocessor with all required parameters, including the driver
processor = DataPreprocessor(server, database, username, password, driver)


print(os.getcwd())


hej
/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/Behavior_Scorecard-DATA-223/2. Code/Classes


In [2]:


path = "../../1. Data/Loan Portfolio Deli v2.sql"


df = processor.fetch_data_from_sql(path)



In [3]:
df.CoappFlag.isna()

0         True
1         True
2         True
3         True
4         True
         ...  
56193    False
56194    False
56195    False
56196    False
56197    False
Name: CoappFlag, Length: 56198, dtype: bool

In [4]:
df['CoappFlag'] = np.where((df['CoappFlag'] != 0) & (df['CoappFlag'] != 1) | (df['CoappFlag'].isna()), 0, df['CoappFlag'])

In [5]:

main = df[(df.CoappFlag == 0)]
co = df[df.CoappFlag == 1]

main = main[~main.AccountNumber.isin(co.AccountNumber)]



df = pd.concat([main,co])

df = df[df.AccountStatus.isin(['OPEN','FROZEN','COLLECTION'])]


In [6]:
df

,SnapshotDate,AccountNumber,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,NoKronofogden,...,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months,Score,RiskClass
76,2023-12-31,5401138,FROZEN,168762.99,82.0,2017-02-13,78,0.0,0,1,...,168762.99,168762.99,168762.99,None,0,0,0,0,100.000000,E8
79,2024-01-31,5401138,FROZEN,168762.99,83.0,2017-02-13,78,0.0,0,1,...,168762.99,168762.99,168762.99,None,0,0,0,0,100.000000,E8
83,2024-02-29,5401138,FROZEN,168762.99,84.0,2017-02-13,78,0.0,0,1,...,NaN,NaN,NaN,None,0,0,0,0,100.000000,E8
84,2023-12-31,5058573,FROZEN,227570.96,85.0,2016-11-15,126,0.0,0,1,...,227570.96,227570.96,227570.96,None,0,0,0,0,100.000000,E8
89,2024-01-31,5058573,FROZEN,227570.96,86.0,2016-11-15,126,0.0,0,1,...,227570.96,227570.96,227570.96,None,0,0,0,0,100.000000,E8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56184,2024-02-29,5000385,FROZEN,224655.26,100.0,2015-10-26,123,1.0,0,1,...,NaN,NaN,NaN,None,0,0,0,0,100.000000,E8
56186,2024-01-31,5000179,FROZEN,194394.86,100.0,2015-09-28,73,1.0,0,1,...,388789.72,388789.72,388789.72,None,0,0,0,0,100.000000,E8
56188,2024-02-29,5000054,OPEN,172666.19,101.0,2015-09-16,43,1.0,0,1,...,NaN,NaN,NaN,None,0,0,0,0,1.484099,E3
56189,2024-02-29,5000294,FROZEN,338251.56,101.0,2015-09-25,118,1.0,0,1,...,NaN,NaN,NaN,None,0,0,0,0,100.000000,E8


In [7]:
co

,SnapshotDate,AccountNumber,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,NoKronofogden,...,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months,Score,RiskClass


In [4]:
df[df.AccountNumber == 5058573]

,SnapshotDate,AccountNumber,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,NoKronofogden,...,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months,Score,RiskClass


In [3]:
df['AccountNumber']   = df.AccountNumber.astype(int)

In [5]:
df

,SnapshotDate,AccountNumber,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,NoKronofogden,...,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months,Score,RiskClass


In [4]:

################### IMPORT MACRO INSTRUMENT DATA ##########################


path = "../../1. Data"
os.chdir(path)

MacroInstrument = pd.read_excel('Macro_Instrument.xlsx')

#df.to_csv('BSC_Today.csv', index=False)

MacroInstrument = MacroInstrument[['Date','Instrument Rolling Mean']]
MacroInstrument['Instrument Rolling Mean'] = np.where(MacroInstrument['Instrument Rolling Mean'].notna(), MacroInstrument['Instrument Rolling Mean'],1 )    ## Will have 1 if NAN but this shall be updated each month 
MacroInstrument['Date'] =  MacroInstrument['Date'].astype(str)
MacroInstrument






#######################   CALCULATE BEHVAIOUR MODEL         ##################################
print('hej1')

coefficients = np.array([-0.44414603,  0.18778622 , 0.3539554 ,  0.70178643])
intercept = np.array([0])


# Sample DataFrame (Assuming you already have this in place)
pd_ = df.copy()


# Compute the Z values using your logistic regression model
pd_['Z'] = (pd_.CoappFlag * coefficients[0] +
            pd_.Ever30In6Months * coefficients[1] + 
            pd_.WorstDelinquency6M * coefficients[2] +
            pd_.CurrentDelinquencyStatus * coefficients[3] +

            intercept[0]) 

# Compute the original probabilities
pd_['P'] = pd_['Z'].apply(lambda x: 1 / (1 + np.exp(-x)))

# Coefficients and Intercept from the Calibration model
calibration_coef = 9.86439545
calibration_intercept = -8.57437634


print("Coefficient:", calibration_coef)
print("Intercept:", calibration_intercept)

# Using the original probabilities to calibrate them with the calibration model
pd_['Z_calibrated'] = pd_['P'].apply(lambda x: x * calibration_coef + calibration_intercept)

# Compute the calibrated probabilities
pd_['BehaviourModel'] = pd_['Z_calibrated'].apply(lambda x: 1 / (1 + np.exp(-x)))


pd_ = pd_[['AccountNumber','AccountStatus','SnapshotDate',	'MOB'	,'DisbursedDate',	'CurrentAmount','RemainingTenor','CoappFlag',	'Ever30In6Months',	'WorstDelinquency6M','CurrentDelinquencyStatus','WorstDelinquency12M','Ever30In12Months','Ever90In12Months'	,'Score'	,'RiskClass','P','BehaviourModel','Ever90','ForberanceIn6Months','ForberanceIn12Months']]
pd_.loc[:, 'DisbursedDate'] = pd.to_datetime(pd_['DisbursedDate'])


BehaviourDone = pd_.copy()





print('hej2')
#######################         CALCULATE ADMISSION MODEL         ##################################

main_path = "../1. Data/MA Correct join - APL CRB-MLP Today.sql"
co_path = "../1. Data/CO Min score join - APL CBR MLP Today.sql"

preprocessor = DataPreprocessor(server, database, username, password,driver)
final_df = preprocessor.process_data(main_path, co_path)


pd_ = final_df[['SSN','PDScoreNew','UCScore','age' ,'Inquiries12M','UtilizationRatio','Amount','MaritalStatus','ReceivedDate','DisbursedDate','Applicationtype','Ever90',
                'Ever30',
                'AccountNumber','CapitalDeficit','PropertyVolume','PaymentRemarks','IndebtednessRatio','ApplicationScore', 'StartupFee','PaymentRemarksNo'] ]





# Assuming pd_ is your DataFrame and it's already defined

# Get the current date
now = dt.datetime.now()

# Get the first day of the current month
first_day_of_month = dt.datetime(now.year, now.month, 1)

# Ensure 'DisbursedDate' is in datetime format if it's not already
pd_.loc[:, 'DisbursedDate'] = pd.to_datetime(pd_['DisbursedDate'])


# Filter the DataFrame for rows where 'DisbursedDate' is less than the first day of the current month
pd_ = pd_[pd_['DisbursedDate'] < first_day_of_month]

# Print the maximum 'DisbursedDate' from the filtered DataFrame
print(pd_['DisbursedDate'].max())



print('hej3')


# Coefficients and Intercept from the Logistic Regression model
coefficients = np.array([2.03675292e+00 ,-2.18071234e-02  ,3.39715771e-02, -2.12322589e-07])  
intercept = np.array([-0.13407141])

print("Coefficients:", coefficients)
print("Intercept:", intercept)


# Compute the Z values using your logistic regression model
pd_['Z'] = (pd_.UCScore * coefficients[0] +
            pd_.age * coefficients[1] + 
            pd_.Inquiries12M * coefficients[2] +
            pd_.PropertyVolume * coefficients[3] + 
            intercept[0]) 

# Compute the original probabilities
pd_['P'] = pd_['Z'].apply(lambda x: 1 / (1 + np.exp(-x)))

# Coefficients and Intercept from the Calibration model
calibration_coef = 3.7812065422080856
calibration_intercept = -4.336067082588543




# Using the original probabilities to calibrate them with the calibration model
pd_['Z_calibrated'] = pd_['P'].apply(lambda x: x * calibration_coef + calibration_intercept)

# Compute the calibrated probabilities
pd_['AdmissionModel'] = pd_['Z_calibrated'].apply(lambda x: 1 / (1 + np.exp(-x)))


AdmissionDone = pd_[['AccountNumber','PDScoreNew','UCScore','age','Inquiries12M','PropertyVolume','AdmissionModel','ApplicationScore']]
AdmissionDone['AccountNumber'] = AdmissionDone['AccountNumber'] #.astype(int)

print(type(AdmissionDone['AccountNumber']))
print(type(BehaviourDone['AccountNumber']))


print('hej4')
#######################         CREATE SICR LOGIC         ##################################


AdmissionDone = pd_[['AccountNumber','PDScoreNew','UCScore','age','Inquiries12M','PropertyVolume','AdmissionModel','ApplicationScore']]
AdmissionDone['AccountNumber'] = AdmissionDone['AccountNumber'] # .astype(int)

together = pd.merge(BehaviourDone,AdmissionDone , on='AccountNumber', how='left')


## Only OPEN & FROZEN ACCOUNTS
lek = together[ (together.SnapshotDate >= min(together.SnapshotDate) )]



# Ensure DisbursedDate is a datetime object (if not already)
lek['DisbursedDate'] = pd.to_datetime(lek['DisbursedDate'])

# Convert the string to a datetime object
comparison_date = pd.to_datetime('2023-12-20')



lek['AppliedApplicationScore'] = np.where(
    (lek['DisbursedDate'] > comparison_date) &
    (np.round(lek['PDScoreNew'], 2) <= np.round(lek['AdmissionModel'], 2)) &
    (lek['PDScoreNew'].notna()), 
    lek['PDScoreNew'],  

    np.where(
        (lek['DisbursedDate'] > comparison_date) &
        
        lek['PDScoreNew'].isna(),  
        lek['AdmissionModel'],  

        np.where(
            (lek['DisbursedDate'] <= comparison_date), 
            lek['ApplicationScore'] / 100,  
            lek['AdmissionModel'] 
        )
    )
)

lek['AdjustedBehaviourScore'] = np.where(  lek['CurrentDelinquencyStatus'].isin([4,9]) ,1.0 , lek['BehaviourModel'])






see = lek.copy()


see['AppliedApplicationScore'] = np.where(  see.AppliedApplicationScore.isna()   ,0 , see.AppliedApplicationScore )


see['PD_Delta'] = see.AdjustedBehaviourScore - see.AppliedApplicationScore 


see = see[see.MOB.notna()]   ## take away accounts that was closed last month

see['PD_Delta'] = np.where(see['PD_Delta'].isna() , 0,see['PD_Delta'])

see = see.sort_values(by='PD_Delta')


see['FBE'] = np.where( (see.ForberanceIn12Months == 1) &  (see.CurrentDelinquencyStatus > 1) , 1,0)


see['SICR'] = np.where((see.PD_Delta > 0.0675) | (see['FBE'] == 1), 1, 0)



## Apply a lifetime factor, this is based from UCBLANCO VINTAGE ANALYSIS, in lower risk but still high 20 % increase and on the rest it will be 10 % increase

see['AdjustedBehaviourScore'] = np.where( (see['SICR'] == 1) &(see['AdjustedBehaviourScore'] < 0.50) , see.AdjustedBehaviourScore * 1.2 , 
                      np.where( (see['SICR'] == 1) &(see['AdjustedBehaviourScore'] >= 0.50) , see.AdjustedBehaviourScore * 1.1 ,   see.AdjustedBehaviourScore )) ## Adding LifeTime Convertion to Stage 2 


see['AdjustedBehaviourScore'] = np.where( see.AdjustedBehaviourScore > 1,1,see.AdjustedBehaviourScore)



see['Stageing'] = np.where(   (see['SICR'] == 0 ) 
                           
                           ,'Stage1',
                           np.where(   see['AdjustedBehaviourScore'] == 1.0 ,'Stage3','Stage2'))


see = see.drop_duplicates()


# Merge lek with MacroInstrument on 'SnapshotDate' in lek and 'Date' in MacroInstrument
see = pd.merge(see, MacroInstrument, left_on='SnapshotDate', right_on='Date', how='left')

see['AdjustedBehaviourScore'] =  see['BehaviourModel'] * see['Instrument Rolling Mean']

see['AdjustedBehaviourScore'] = np.where(  see['CurrentDelinquencyStatus'].isin([4,9]) ,1.0 , see['BehaviourModel'])

hej1
Coefficient: 9.82696528
Intercept: -8.57437634
hej2
2024-02-29 00:00:00
hej3
Coefficients: [ 2.03675292e+00 -2.18071234e-02  3.39715771e-02 -2.12322589e-07]
Intercept: [-0.13407141]
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
hej4


In [15]:
see[see.AccountNumber == 5058573]

#5458583 , 5401138 ,5058573

,AccountNumber,AccountStatus,SnapshotDate,MOB,DisbursedDate,CurrentAmount,RemainingTenor,CoappFlag,Ever30In6Months,WorstDelinquency6M,...,AdmissionModel,ApplicationScore,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing,Date,Instrument Rolling Mean
